In [41]:
from fastembed import TextEmbedding

Q1. Embedding the query

Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

You should get a numpy array of size 512.

What's the minimal value in this array?

-0.51

-0.11

0

0.51

In [42]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
embedder = TextEmbedding(model_name=model_handle)

sentence = "I just discovered the course. Can I join now?"
embedding = next(embedder.embed([sentence]))

print(len(embedding))
print(min(embedding))

512
-0.11726373885183883


In [43]:
import numpy as np
np.linalg.norm(embedding)


np.float64(1.0)

In [44]:
embedding.dot(embedding)

np.float64(1.0000000000000002)

Q2. Cosine similarity with another vector

Now let's embed this document:

doc = 'Can I still join the course after the start date?'

In [45]:
sentence2 = "Can I still join the course after the start date?"
embedding2 = next(embedder.embed([sentence2]))

embedding.dot(embedding2)

np.float64(0.9008528895674548)

Q3. Ranking by cosine

For Q3 and Q4 we will use these documents.

In [46]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

0

1

2

3

4

Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:

V.dot(q)

In [47]:
documents_to_embed = [document["text"] for document in documents]
embeddings3 = list(embedder.embed(documents_to_embed))
embeddings3_matrix = np.asarray(embeddings3)
np.argmax(embeddings3_matrix.dot(embedding))

np.int64(1)

Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of question and text:

full_text = doc['question'] + ' ' + doc['text']

Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

0

1

2

3

4

Is it different from Q3? If yes, why?

In [48]:
documents_to_embed = [document['question'] + ' ' + document['text'] for document in documents]
embeddings4 = list(embedder.embed(documents_to_embed))
embeddings4_matrix = np.asarray(embeddings4)
np.argmax(embeddings4_matrix.dot(embedding))

np.int64(0)

Q5. Selecting the embedding model

Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

128

256

384

512

One of these models is BAAI/bge-small-en. Let's use it.

In [49]:
dims = []
for model in TextEmbedding.list_supported_models():
    dims.append(model["dim"])
min(dims)

384

In [50]:
model_handle = "BAAI/bge-small-en"

Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp.

In [51]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

text = doc['question'] + ' ' + doc['text']

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results? (The score for the first returned record):

0.97

0.87

0.77

0.67

In [52]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://qdrant:6333")  # connecting to local Qdrant instance

In [53]:
EMBEDDING_DIMENSIONALITY = min(dims)

In [54]:
# Define the collection name
collection_name = "zoomcamp-hw2"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [55]:
points = []
id = 0

for doc in documents:

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_handle),
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        }  # save all needed metadata fields
    )
    points.append(point)

    id += 1

In [56]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [61]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

results = search(sentence)

In [69]:
results.points[0].score

0.8703172